# <center>Weight Data

Objective of this notebook is to bring in all the data related to my health:

1. Weighing scale data
2. MyFitness Pal
3. Garmin watch
4. Samsung Health 

In [1]:
import pandas as pd
import numpy as np
import os
import requests

os.chdir('/Users/jayrajparmar/Documents/side_project/health_data_tracking')

import basic_methods as bm

/usr/local/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
xml_data = """<?xml version='1.0' encoding='utf-8'?>
            <Records count="7488">
                <Record contentId="1002280" levelId="309" levelGuid="xyz-xyz-xyz" moduleId="511" parentId="0">
                    <Record contentId="5538135" levelId="304" levelGuid="xyz1-xyz1-xyz1" moduleId="506" parentId="0">
                         <Field id="22028" guid="abc1-abc1-abc1" type="4">CP-1002280</Field>
                            <ListValues>
                                <ListValues id="74343" displayName="No">No</ListValues>
                            </ListValues>
                            </Record>
                    <Record contentId="6217826" levelId="304" levelGuid="xyz2-xyz2-xyz2" moduleId="506" parentId="0">
                         <Field id="22028" guid="abc1-abc1-abc1" type="4">CP-1002280</Field>
                            <ListValues>
                                <ListValues id="74343" displayName="No">No</ListValues>
                            </ListValues></Record>
                    </Record>
    <Field id="20518" guid="abc-abc-abc" type="1">CP-1002280</Field>
    </Records>"""


In [25]:
pd.read_xml(xml_data,xpath='//Record', elems_only=True)


# //x:record

,Record,Field,ListValues
0,NaN,None,NaN
1,NaN,CP-1002280,NaN
2,NaN,CP-1002280,NaN


In [3]:
import pandas_read_xml as pdx
from pandas_read_xml import fully_flatten

In [11]:
pdx.read_xml(xml_data,['Records', 'Record','Record']).pipe(fully_flatten)

,0|@contentId,0|@levelId,0|@levelGuid,0|@moduleId,0|@parentId,1|@contentId,1|@levelId,1|@levelGuid,1|@moduleId,1|@parentId,0|Field|@id,0|Field|@guid,0|Field|@type,0|Field,1|Field|@id,1|Field|@guid,1|Field|@type,1|Field,0|ListValues|ListValues|@id,0|ListValues|ListValues|@displayName,0|ListValues|ListValues,1|ListValues|ListValues|@id,1|ListValues|ListValues|@displayName,1|ListValues|ListValues
0,5538135,304,xyz1-xyz1-xyz1,506,0,6217826,304,xyz2-xyz2-xyz2,506,0,22028,abc1-abc1-abc1,4,CP-1002280,22028,abc1-abc1-abc1,4,CP-1002280,74343,No,No,74343,No,No


In [23]:
response = requests.get('https://drive.google.com/drive/folders/1HH0K5I3KLmNdxbwI8Px-su0Kdfn0sx4I')
assert response.status_code == 200, 'Wrong status code'
print(response.content)

b'\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n  <meta charset="utf-8">\n  <meta content="width=300, initial-scale=1" name="viewport">\n  <meta name="description" content="Google Drive is a free way to keep your files backed up and easy to reach from any phone, tablet, or computer. Start with 15GB of Google storage \xe2\x80\x93 free.">\n  <meta name="google-site-verification" content="LrdTUW9psUAMbh4Ia074-BPEVmcpBxF6Gwf0MSgQXZs">\n  <title>Meet Google Drive \xe2\x80\x93 One place for all your files</title>\n  <style>\n  @font-face {\n  font-family: \'Open Sans\';\n  font-style: normal;\n  font-weight: 300;\n  src: url(//fonts.gstatic.com/s/opensans/v15/mem5YaGs126MiZpBA-UN_r8OUuhs.ttf) format(\'truetype\');\n}\n@font-face {\n  font-family: \'Open Sans\';\n  font-style: normal;\n  font-weight: 400;\n  src: url(//fonts.gstatic.com/s/opensans/v15/mem8YaGs126MiZpBA-UFVZ0e.ttf) format(\'truetype\');\n}\n  </style>\n  <style>\n  h1, h2 {\n  -webkit-animation-duration: 0.1s;\n  -webkit-anim

In [26]:
str(response.content).split('\n')

['b\'\\n<!DOCTYPE html>\\n<html lang="en">\\n  <head>\\n  <meta charset="utf-8">\\n  <meta content="width=300, initial-scale=1" name="viewport">\\n  <meta name="description" content="Google Drive is a free way to keep your files backed up and easy to reach from any phone, tablet, or computer. Start with 15GB of Google storage \\xe2\\x80\\x93 free.">\\n  <meta name="google-site-verification" content="LrdTUW9psUAMbh4Ia074-BPEVmcpBxF6Gwf0MSgQXZs">\\n  <title>Meet Google Drive \\xe2\\x80\\x93 One place for all your files</title>\\n  <style>\\n  @font-face {\\n  font-family: \\\'Open Sans\\\';\\n  font-style: normal;\\n  font-weight: 300;\\n  src: url(//fonts.gstatic.com/s/opensans/v15/mem5YaGs126MiZpBA-UN_r8OUuhs.ttf) format(\\\'truetype\\\');\\n}\\n@font-face {\\n  font-family: \\\'Open Sans\\\';\\n  font-style: normal;\\n  font-weight: 400;\\n  src: url(//fonts.gstatic.com/s/opensans/v15/mem8YaGs126MiZpBA-UFVZ0e.ttf) format(\\\'truetype\\\');\\n}\\n  </style>\\n  <style>\\n  h1, h2 {\\n 

In [4]:
def download_file_from_google_drive(id, destination):
    URL = "https://drive.google.com/drive/folders/1HH0K5I3KLmNdxbwI8Px-su0Kdfn0sx4I"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

if __name__ == "__main__":
    file_id = '1HH0K5I3KLmNdxbwI8Px-su0Kdfn0sx4I'
    destination = '/Users/jayrajparmar/Documents/side_project/health_data_tracking/temp.csv'
    download_file_from_google_drive(file_id, destination)

In [2]:
df_initial = pd.read_csv('/Users/jayrajparmar/Documents/side_project/health_data_tracking/Weight_data/Measurement Data.csv')
df_initial.head()

,Time of Measurement,Weight(kg),BMI,Body Fat(%),Fat-free Body Weight(kg),Subcutaneous Fat(%),Visceral Fat,Body Water(%),Skeletal Muscle(%),Muscle Mass(kg),Bone Mass(kg),Protein(%),BMR(kcal),Metabolic Age,Remarks
0,"April 9, 2022 09:28:03",80.9000,28.3000,16.01,67.95,13.1,10.0,60.6,54.3,64.50,3.40,19.1,1837.0,28.0,--
1,"April 8, 2022 06:19:05",81.6500,28.6000,16.19,68.43,13.2,10.0,60.5,54.1,65.00,3.42,19.1,1848.0,29.0,--
2,"April 7, 2022 06:08:45",81.6500,28.6000,16.22,68.41,13.2,10.0,60.5,54.1,65.00,3.42,19.1,1847.0,29.0,--
3,"April 6, 2022 06:14:36",81.2500,28.4000,16.08,68.18,13.1,10.0,60.6,54.2,64.80,3.41,19.1,1842.0,28.0,--
4,"April 5, 2022 06:04:17",81.4000,28.5000,16.13,68.27,13.2,10.0,60.6,54.2,64.90,3.41,19.1,1844.0,29.0,--


In [ ]:
bm.datetime_conversion(df_initial, 'Time of Measurement')

def datetime_conversion(df,col_list):
    df[col_list] = df[col_list].apply(pd.to_datetime, errors = 'coerce')
    

In [8]:
def data_prep(df):
    # Column name cleaning
    df.columns = [i.replace(' ','_').replace('(','').replace(')','') for i in list(df_initial)]
    print(df.columns)
    df = bm.datetime_conversion(df, 'Time_of_Measurement')
    return df
    

In [9]:
df_test = data_prep(df_initial)
df_test

Index(['Time_of_Measurement', 'Weightkg', 'BMI', 'Body_Fat%',
       'Fat-free_Body_Weightkg', 'Subcutaneous_Fat%', 'Visceral_Fat',
       'Body_Water%', 'Skeletal_Muscle%', 'Muscle_Masskg', 'Bone_Masskg',
       'Protein%', 'BMRkcal', 'Metabolic_Age', 'Remarks'],
      dtype='object')


In [149]:
# Column name cleaning
df_initial.columns = [i.replace(' ','_').replace('(','').replace(')','') for i in list(df_initial)]

# Getting index as the date from time of Measurement column

df_initial['Time_of_Measurement'] = pd.to_datetime(df_initial.Time_of_Measurement)
df_initial.index = df_initial['Time_of_Measurement'].dt.date

# Converting all the columns to numeric 
lis = []
for i in df_initial.columns:
    if df_initial[i].dtypes != 'datetime64[ns]':
        lis.append(df_initial[i].apply(pd.to_numeric, errors='coerce'))
        print(str(i)+str('(')+str(df_initial[i].dtypes)+str(')')+str(':')+'Done')
        
    else: print('nothing')
df_dtyp = pd.concat(lis, axis=1)
df_dtyp = df_dtyp.drop_duplicates()
del df_dtyp['Remarks']
df_dtyp = df_dtyp.dropna(axis='index')
df_initial.index.names = ['Dates']
df_final = df_dtyp.merge(pd.DataFrame(df_initial.groupby(['Dates'])['Time_of_Measurement'].max()),
                         how='left',left_index=True, right_index=True)
cols = ['Weightkg', 'BMI', 'Body_Fat%', 'Fat-free_Body_Weightkg',
       'Subcutaneous_Fat%', 'Visceral_Fat', 'Body_Water%', 'Skeletal_Muscle%',
       'Muscle_Masskg', 'Bone_Masskg', 'Protein%', 'BMRkcal', 'Metabolic_Age']
agg_func = ['max']*13

d = dict(zip(cols,agg_func))
df_dedup = df_final.groupby(['Time_of_Measurement']).agg(d)
df_dedup['Dates'] = df_dedup.index.date
print('*****************************Datatypes after conversion*****************************')
print(df_final.dtypes)
# df_final = df_dup.dropna(axis='index')

print(df_dtyp.shape)
print(df_final.shape)
print(df_dedup.shape)
# Generating 7 days rolling average for all the health indicators








nothing
Weightkg(float64):Done
BMI(float64):Done
Body_Fat%(object):Done
Fat-free_Body_Weightkg(object):Done
Subcutaneous_Fat%(object):Done
Visceral_Fat(object):Done
Body_Water%(object):Done
Skeletal_Muscle%(object):Done
Muscle_Masskg(object):Done
Bone_Masskg(object):Done
Protein%(object):Done
BMRkcal(object):Done
Metabolic_Age(object):Done
Remarks(object):Done
*****************************Datatypes after conversion*****************************
Weightkg                         float64
BMI                              float64
Body_Fat%                        float64
Fat-free_Body_Weightkg           float64
Subcutaneous_Fat%                float64
Visceral_Fat                     float64
Body_Water%                      float64
Skeletal_Muscle%                 float64
Muscle_Masskg                    float64
Bone_Masskg                      float64
Protein%                         float64
BMRkcal                          float64
Metabolic_Age                    float64
Time_of_Measurement

In [150]:
df_dedup.sort_values()

,Weightkg,BMI,Body_Fat%,Fat-free_Body_Weightkg,Subcutaneous_Fat%,Visceral_Fat,Body_Water%,Skeletal_Muscle%,Muscle_Masskg,Bone_Masskg,Protein%,BMRkcal,Metabolic_Age,Dates
Time_of_Measurement,,,,,,,,,,,,,,
2021-02-25 18:05:23,88.4000,31.0000,18.0000,72.4900,14.5000,13.0000,59.2000,53.0000,68.9000,3.6200,18.7000,"1,935.0000",29.0000,2021-02-25
2021-04-04 22:44:21,89.0500,31.2000,18.1600,72.8700,14.6000,13.0000,65.2000,58.3000,69.2000,3.6400,20.6000,"1,944.0000",29.0000,2021-04-04
2021-04-08 07:32:02,87.7500,30.7000,17.8100,72.1200,14.3000,12.0000,59.3000,53.1000,68.5000,3.6100,18.7000,"1,927.0000",29.0000,2021-04-08
2021-04-09 08:54:00,87.2500,30.5000,17.7000,71.8100,14.3000,12.0000,59.4000,53.2000,68.2000,3.5900,18.8000,"1,921.0000",29.0000,2021-04-09
2021-04-10 09:52:52,88.0500,30.8000,17.8800,72.3100,14.4000,13.0000,59.3000,53.1000,68.7000,3.6200,18.7000,"1,931.0000",29.0000,2021-04-10
2021-04-13 07:09:50,88.8000,31.1000,18.1100,72.7200,14.6000,13.0000,59.1000,52.9000,69.1000,3.6400,18.7000,"1,940.0000",29.0000,2021-04-13
2021-04-14 07:43:21,88.4500,31.0000,18.0100,72.5200,14.5000,13.0000,59.2000,53.0000,68.9000,3.6300,18.7000,"1,936.0000",29.0000,2021-04-14
2021-04-15 07:40:37,88.2500,30.9000,17.9600,72.4000,14.5000,13.0000,59.2000,53.0000,68.8000,3.6200,18.7000,"1,933.0000",29.0000,2021-04-15
2021-04-16 07:47:05,88.3000,30.9000,17.9600,72.4400,14.5000,13.0000,59.2000,53.0000,68.8000,3.6200,18.7000,"1,934.0000",29.0000,2021-04-16


In [151]:
df_initial

,Time_of_Measurement,Weightkg,BMI,Body_Fat%,Fat-free_Body_Weightkg,Subcutaneous_Fat%,Visceral_Fat,Body_Water%,Skeletal_Muscle%,Muscle_Masskg,Bone_Masskg,Protein%,BMRkcal,Metabolic_Age,Remarks
Dates,,,,,,,,,,,,,,,
2022-04-09,2022-04-09 09:28:03,80.9000,28.3000,16.01,67.95,13.1,10.0,60.6,54.3,64.50,3.40,19.1,1837.0,28.0,--
2022-04-08,2022-04-08 06:19:05,81.6500,28.6000,16.19,68.43,13.2,10.0,60.5,54.1,65.00,3.42,19.1,1848.0,29.0,--
2022-04-07,2022-04-07 06:08:45,81.6500,28.6000,16.22,68.41,13.2,10.0,60.5,54.1,65.00,3.42,19.1,1847.0,29.0,--
2022-04-06,2022-04-06 06:14:36,81.2500,28.4000,16.08,68.18,13.1,10.0,60.6,54.2,64.80,3.41,19.1,1842.0,28.0,--
2022-04-05,2022-04-05 06:04:17,81.4000,28.5000,16.13,68.27,13.2,10.0,60.6,54.2,64.90,3.41,19.1,1844.0,29.0,--
2022-04-04,2022-04-04 06:34:17,82.8000,29.0000,16.52,69.12,13.4,11.0,60.3,53.9,65.70,3.46,19.0,1863.0,29.0,--
2022-04-03,2022-04-03 08:42:29,81.9500,28.7000,16.28,68.61,13.3,11.0,60.4,54.1,65.20,3.43,19.1,1851.0,29.0,--
2022-04-03,2022-04-03 08:42:15,81.9500,28.7000,16.28,68.61,13.3,11.0,60.4,54.1,65.20,3.43,19.1,1851.0,29.0,--
2022-04-02,2022-04-02 08:44:51,81.4500,28.5000,16.16,68.29,13.2,10.0,60.5,54.2,64.90,3.41,19.1,1844.0,29.0,--
